In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
from scipy.io.matlab.mio import loadmat, savemat
import numpy as np
from PIL import Image
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import shutil
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from sklearn import preprocessing


C:\Users\xia\AppData\Local\Temp\ipykernel_4220\1157685092.py:8: DeprecationWarning: Please use `loadmat` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio` namespace is deprecated.
  from scipy.io.matlab.mio import loadmat, savemat
C:\Users\xia\AppData\Local\Temp\ipykernel_4220\1157685092.py:8: DeprecationWarning: Please use `savemat` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio` namespace is deprecated.
  from scipy.io.matlab.mio import loadmat, savemat


In [36]:
def getData(filename, remove_duplicata = True):
    with open(filename) as f:
        df = pd.read_csv(f)
        if remove_duplicata:
            df = df.drop_duplicates()
        X = df.to_numpy()
    return X[:,:-1], X[:,-1]


In [37]:
X, Y = getData("dataSelection.csv")
np.unique(Y)

array(['selectAll', 'selectL', 'selectR', 'selectWR'], dtype=object)

In [38]:

le = preprocessing.LabelEncoder()

tmp = X[:,X.shape[1]//2:]
anotherX = X[:,:X.shape[1]//2]-tmp
anotherX = anotherX.astype('int64')
#anotherX = normalize(anotherX)
le.fit(Y)
Ynum = le.transform(Y)

In [39]:
dataset = torch.utils.data.TensorDataset(torch.Tensor(anotherX),torch.Tensor(Ynum).type(torch.LongTensor))
dataset[1]

(tensor([-783.,    0.,   -1.,   -4.,   -9.]), tensor(0))

In [40]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_tanH_stack = nn.Sequential(
            nn.Linear(5, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, len(np.unique(Y)))
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_tanH_stack(x)
        return logits


In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [43]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [44]:
epochs = 200

for i in range(5):
    model = NeuralNetwork().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    print("CrossValidation : "+str(i)+"\n-------------------------------\n")
    train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)


    # Create the data loaders for training and validation
    train_dataloader = torch.utils.data.DataLoader(
        train_data,
        batch_size=1,
        shuffle=True
    )

    test_dataloader = torch.utils.data.DataLoader(
        test_data,
        batch_size=1,
        shuffle=False
    )
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        #test(test_dataloader, model, loss_fn)
        test(test_dataloader, model, loss_fn)
    print("Done!")

CrossValidation : 0
-------------------------------

Epoch 1
-------------------------------
loss: 1.355906  [    1/ 3296]
loss: 0.838001  [  101/ 3296]
loss: 1.324145  [  201/ 3296]
loss: 1.587722  [  301/ 3296]
loss: 1.320586  [  401/ 3296]
loss: 1.306448  [  501/ 3296]
loss: 1.573415  [  601/ 3296]
loss: 1.516358  [  701/ 3296]
loss: 1.036168  [  801/ 3296]
loss: 1.527487  [  901/ 3296]
loss: 1.503439  [ 1001/ 3296]
loss: 0.000000  [ 1101/ 3296]
loss: 1.563835  [ 1201/ 3296]
loss: 1.454279  [ 1301/ 3296]
loss: 0.803533  [ 1401/ 3296]
loss: 1.605122  [ 1501/ 3296]
loss: 1.158620  [ 1601/ 3296]
loss: 0.000005  [ 1701/ 3296]
loss: 1.527503  [ 1801/ 3296]
loss: 0.975378  [ 1901/ 3296]
loss: 0.000000  [ 2001/ 3296]
loss: 0.097202  [ 2101/ 3296]
loss: 1.127797  [ 2201/ 3296]
loss: 0.759515  [ 2301/ 3296]
loss: 1.422381  [ 2401/ 3296]
loss: 1.782738  [ 2501/ 3296]
loss: 0.626416  [ 2601/ 3296]
loss: 1.438238  [ 2701/ 3296]
loss: 1.439865  [ 2801/ 3296]
loss: 1.401365  [ 2901/ 3296]
loss: 1

loss: 0.054452  [ 1801/ 3296]
loss: 0.278739  [ 1901/ 3296]
loss: 0.176567  [ 2001/ 3296]
loss: 1.417716  [ 2101/ 3296]
loss: 1.036903  [ 2201/ 3296]
loss: 0.697073  [ 2301/ 3296]
loss: 2.059333  [ 2401/ 3296]
loss: 1.040936  [ 2501/ 3296]
loss: 0.116423  [ 2601/ 3296]
loss: 0.636410  [ 2701/ 3296]
loss: 2.064877  [ 2801/ 3296]
loss: 0.000000  [ 2901/ 3296]
loss: 0.566903  [ 3001/ 3296]
loss: 0.187568  [ 3101/ 3296]
loss: 0.258840  [ 3201/ 3296]
Test Error: 
 Accuracy: 69.8%, Avg loss: 0.651522 

Epoch 9
-------------------------------
loss: 0.262958  [    1/ 3296]
loss: 0.167101  [  101/ 3296]
loss: 1.542018  [  201/ 3296]
loss: 1.679717  [  301/ 3296]
loss: 1.748566  [  401/ 3296]
loss: 1.385269  [  501/ 3296]
loss: 0.600202  [  601/ 3296]
loss: 0.958960  [  701/ 3296]
loss: 0.772909  [  801/ 3296]
loss: 0.790300  [  901/ 3296]
loss: 0.000000  [ 1001/ 3296]
loss: 0.943702  [ 1101/ 3296]
loss: 0.604659  [ 1201/ 3296]
loss: 1.086888  [ 1301/ 3296]
loss: 0.402440  [ 1401/ 3296]
loss: 0.

loss: 0.000000  [  301/ 3296]
loss: 0.145567  [  401/ 3296]
loss: 0.570648  [  501/ 3296]
loss: 0.678440  [  601/ 3296]
loss: 0.055141  [  701/ 3296]
loss: 0.693052  [  801/ 3296]
loss: 0.201043  [  901/ 3296]
loss: 0.451609  [ 1001/ 3296]
loss: 0.554605  [ 1101/ 3296]
loss: 0.705983  [ 1201/ 3296]
loss: 0.000000  [ 1301/ 3296]
loss: 2.620073  [ 1401/ 3296]
loss: 1.757338  [ 1501/ 3296]
loss: 0.648105  [ 1601/ 3296]
loss: 0.611096  [ 1701/ 3296]
loss: 2.589902  [ 1801/ 3296]
loss: 1.222552  [ 1901/ 3296]
loss: 0.982085  [ 2001/ 3296]
loss: 0.000000  [ 2101/ 3296]
loss: 0.075291  [ 2201/ 3296]
loss: 0.689745  [ 2301/ 3296]
loss: 0.552533  [ 2401/ 3296]
loss: 0.000000  [ 2501/ 3296]
loss: 0.000000  [ 2601/ 3296]
loss: 1.328679  [ 2701/ 3296]
loss: 0.031548  [ 2801/ 3296]
loss: 0.727865  [ 2901/ 3296]
loss: 0.003797  [ 3001/ 3296]
loss: 2.363805  [ 3101/ 3296]
loss: 0.003421  [ 3201/ 3296]
Test Error: 
 Accuracy: 75.5%, Avg loss: 0.699780 

Epoch 17
-------------------------------
loss: 1

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#Used to calculate the confusion matrix
y_true = []
y_pred = []
for X,y in test_dataloader:
    y_true.extend(y.data.cpu().numpy())

    X, y = X.to(device), y.to(device)
    pred = model(X)
    y_pred.extend(pred.argmax(1).data.cpu().numpy())
    
conf_matrix = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=np.unique(Y))
disp.plot()
plt.show()
